In [2]:
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = '18'
plt.rcParams['figure.dpi'] = 125
plt.rcParams['figure.facecolor'] = 'white'

import numpy as np
import xarray as xr
import pandas as pd
import nctoolkit as nc
import tqdm as tqd
import requests

IndexError: list index out of range

In [ ]:
year = 2021
url = 'http://tds.maracoos.org/thredds/dodsC/AVHRR/'+str(year)+'/1Agg'
r = requests.get(url, allow_redirects=True)


In [ ]:
print(r.headers.get('content-type'))

ModuleNotFoundError: No module named 'cdo'

In [ ]:
import nctoolkit as nc
import xarray as xr
ds = nc.open_url("https://ruc.noaa.gov/raobs/GetRaobs.cgi?shour=All+Times&ltype=All+Levels&wunits=Knots&bdate=2010052612&edate=2010052715&access=WMO+Station+Identifier&view=NO&StationIDs=72632&osort=Station+Series+Sort&oformat=NetCDF+format+%28Binary%29")
ds_xr = ds.to_xarray()